## Setup and Dependencies

This section prepares the development environment by installing all necessary libraries and tools for our AI project. We'll install three main categories of packages:

1. **Core AI/ML Libraries** - For natural language processing, text embeddings, and model handling
2. **Speech Processing Tools** - For speech recognition and text-to-speech conversion
3. **Audio Utilities** - For audio file processing and manipulation


In [3]:
print(" Starting installation...\n")

# Core AI/ML
print("1️⃣ Core AI & NLP...")
!pip install -q datasets sentence-transformers faiss-cpu transformers accelerate peft langchain-text-splitters

# Speech Processing
print("2️⃣ Speech Recognition & Synthesis...")
!pip install -q faster-whisper Coqui-TTS

# Audio Utilities
print("3️⃣ Audio Processing...")
!pip install -q soundfile librosa pydub js2py

print("\n Installation complete!\n" + "=" * 60)

 Starting installation...

1️⃣ Core AI & NLP...
2️⃣ Speech Recognition & Synthesis...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 93.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 124.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 33.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 22.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 101.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━

## Import Libraries

This section imports all required libraries and configures the environment:

- **Standard Libraries** - Built-in Python modules for file handling, JSON processing, and system operations
- **Data Processing** - NumPy and Librosa for numerical operations and audio analysis
- **AI Models** - PyTorch and Transformers for loading and running language models
- **NLP Tools** - Sentence transformers for embeddings, FAISS for vector search, and text splitting utilities
- **Audio Processing** - Whisper for speech recognition and TTS for text-to-speech synthesis
- **Colab Integration** - Display utilities and Google Colab specific features

In [4]:
# Standard library imports
import os
import sys
import json
import time
import shutil
import io
import wave
import warnings
from pathlib import Path
from base64 import b64decode

# Data & Scientific
import numpy as np
import librosa

# Deep Learning
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, pipeline

# NLP & Embeddings
from datasets import load_dataset
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss

# Audio Processing
from pydub import AudioSegment
from faster_whisper import WhisperModel
from TTS.api import TTS

# Colab specific imports
from IPython.display import display, HTML, Javascript, Audio
from google.colab import output

# Configuration
warnings.filterwarnings('ignore')

# System info
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

Torch version: 2.9.0+cu126
CUDA available: True


## Data Preparation

In this section, we will prepare the data for our system by:

1. **Loading Wikipedia articles** - We'll fetch 100 articles as a sample dataset
2. **Filtering the text** - We'll keep only articles with sufficient content (more than 50 characters)
3. **Splitting text into chunks** - We'll divide articles into smaller pieces of approximately 600 characters each
4. **Saving the data** - We'll store these chunks in a JSON file for later use

**Goal:** Prepare a small knowledge base that the system can search through and use to answer questions.

In [5]:
# 1. Load Dataset
print("Loading Wikipedia dataset...")

try:
    ds = load_dataset(
        "wikimedia/wikipedia",
        "20231101.en",
        split="train[:100]",
        trust_remote_code=False
    )
except:
    print("Using alternative dataset...")
    ds = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:100]")

# 2. Filter and Combine Text
print("\nFiltering articles...")

# Keep only texts > 50 characters
texts = [x["text"] for x in ds if x.get("text") and len(x["text"]) > 50]

# Combine first 30 articles
full_text = "\n\n".join(texts[:30])
print(f"Combined length: {len(full_text):,} characters")

# 3. Split into Chunks
print("\nSplitting into chunks...")

splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100
)
chunks = splitter.split_text(full_text)
print(f"Number of chunks: {len(chunks)}")

# 4. Save to File
print("\nSaving chunks...")

Path("data").mkdir(exist_ok=True)
with open("data/chunks.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

print("✅ Saved to: data/chunks.json")
print(f"\nFirst chunk preview:\n{chunks[0][:200]}...\n")

Loading Wikipedia dataset...


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

20231101.en/train-00000-of-00041.parquet:   0%|          | 0.00/420M [00:00<?, ?B/s]

20231101.en/train-00001-of-00041.parquet:   0%|          | 0.00/351M [00:00<?, ?B/s]

...

20231101.en/train-00038-of-00041.parquet:   0%|          | 0.00/538M [00:00<?, ?B/s]

20231101.en/train-00039-of-00041.parquet:   0%|          | 0.00/465M [00:00<?, ?B/s]

20231101.en/train-00040-of-00041.parquet:   0%|          | 0.00/422M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6407814 [00:00<?, ? examples/s]


Filtering articles...
Combined length: 1,023,057 characters

Splitting into chunks...
Number of chunks: 2586

Saving chunks...
✅ Saved to: data/chunks.json

First chunk preview:
Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy, typi...



## Vector Store Creation

In this section, we will:
- **Convert text chunks into numerical vectors (embeddings)** - Transform each text piece into numbers that AI can understand
- **Build a searchable index** - Create a FAISS database that allows fast similarity search
- **Save everything** - Store the index and embeddings for later use

**Goal:** Create a vector database that enables quick retrieval of relevant information based on semantic similarity.

In [6]:
# 1. Load Model
print("Loading embedding model...")
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 2. Load Chunks
with open("data/chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)
print(f"Loaded {len(chunks)} chunks")

# 3. Create Embeddings
print("Converting text to vectors...")
embeddings = embedder.encode(
    chunks,
    show_progress_bar=True,
    convert_to_numpy=True
).astype("float32")

# 4. Build Index
print("Building FAISS index...")
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# 5. Save
faiss.write_index(index, "data/rag_index.faiss")
np.save("data/embeddings.npy", embeddings)

print(f"\n✅ Done! Index: {index.ntotal} vectors, dim: {dimension}")

Loading embedding model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loaded 2586 chunks
Converting text to vectors...


Batches:   0%|          | 0/81 [00:00<?, ?it/s]

Building FAISS index...

✅ Done! Index: 2586 vectors, dim: 384


## Audio Input

In this section, we will:
- **Create an audio recording interface** - Build a web-based microphone recorder directly in the notebook
- **Visualize audio in real-time** - Display waveform animation while recording
- **Save recordings** - Convert and save audio as WAV files for speech recognition
- **Enable user interaction** - Allow users to record, download, or upload audio to Colab

**Goal:** Provide an easy way to capture voice input for testing the voice assistant system.

In [8]:
# AUDIO INPUT INTERFACE
print("=" * 60)
print("🎤 Setting Up Audio Recording Interface")
print("=" * 60)


# Audio Recorder UI
display(HTML("""
<style>
  .audio-controls {
    font-family: Arial, sans-serif;
    padding: 20px;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    border-radius: 12px;
    max-width: 500px;
    margin: 10px 0;
  }
  .btn-group {
    display: flex;
    gap: 10px;
    margin-bottom: 15px;
  }
  .record-btn {
    flex: 1;
    padding: 12px 24px;
    border: none;
    border-radius: 8px;
    font-size: 16px;
    font-weight: 600;
    cursor: pointer;
    transition: all 0.3s ease;
  }
  .record-btn:hover:not(:disabled) {
    transform: translateY(-2px);
  }
  .record-btn:disabled {
    opacity: 0.5;
    cursor: not-allowed;
  }
  .start-btn { background: #4CAF50; color: white; }
  .stop-btn { background: #f44336; color: white; }
  .download-btn { background: #2196F3; color: white; }
  .status {
    color: white;
    text-align: center;
    padding: 10px;
    border-radius: 6px;
    background: rgba(255,255,255,0.1);
    margin-top: 10px;
  }
  .timer {
    font-size: 24px;
    font-weight: bold;
    text-align: center;
    color: white;
    margin: 10px 0;
  }
  .waveform {
    height: 60px;
    background: rgba(255,255,255,0.1);
    border-radius: 6px;
    margin: 10px 0;
    display: flex;
    align-items: center;
    justify-content: center;
  }
  .wave-bar {
    width: 3px;
    background: white;
    margin: 0 2px;
    border-radius: 2px;
    transition: height 0.1s;
  }
  .recording { animation: pulse 1.5s infinite; }
  @keyframes pulse {
    0%, 100% { opacity: 1; }
    50% { opacity: 0.7; }
  }
</style>

<div class="audio-controls">
  <div class="btn-group">
    <button class="record-btn start-btn" id="startBtn" onclick="startRecording()">
      🎤 Start Recording
    </button>
    <button class="record-btn stop-btn" id="stopBtn" onclick="stopRecording()" disabled>
      ⏹️ Stop
    </button>
  </div>
  <div class="btn-group">
    <button class="record-btn download-btn" id="downloadBtn" onclick="downloadAudio()" disabled>
      💾 Download
    </button>
    <button class="record-btn download-btn" id="uploadBtn" onclick="uploadToColab()" disabled>
      ☁️ Upload to Colab
    </button>
  </div>
  <div class="timer" id="timer">00:00</div>
  <div class="waveform" id="waveform"></div>
  <div class="status" id="status">Ready to record</div>
</div>

<script>
let mediaRecorder, audioChunks = [], audioBlob;
let startTime, timerInterval, audioContext, analyser, dataArray, animationId;


// Create waveform visualization bars
const waveform = document.getElementById('waveform');
for(let i = 0; i < 20; i++) {
  const bar = document.createElement('div');
  bar.className = 'wave-bar';
  bar.style.height = '5px';
  waveform.appendChild(bar);
}

// Update recording timer
function updateTimer() {
  const elapsed = Math.floor((Date.now() - startTime) / 1000);
  const mins = Math.floor(elapsed / 60).toString().padStart(2, '0');
  const secs = (elapsed % 60).toString().padStart(2, '0');
  document.getElementById('timer').textContent = `${mins}:${secs}`;
}

// Animate waveform based on audio input
function animateWaveform() {
  if(analyser) {
    analyser.getByteFrequencyData(dataArray);
    const bars = document.querySelectorAll('.wave-bar');
    bars.forEach((bar, i) => {
      const value = dataArray[i * 5] || 0;
      const height = Math.max(5, (value / 255) * 50);
      bar.style.height = height + 'px';
    });
  }
  animationId = requestAnimationFrame(animateWaveform);
}

// Start recording audio
async function startRecording() {
  try {
    const stream = await navigator.mediaDevices.getUserMedia({
      audio: { echoCancellation: true, noiseSuppression: true }
    });

    // Setup audio visualization
    audioContext = new AudioContext();
    analyser = audioContext.createAnalyser();
    analyser.fftSize = 256;
    audioContext.createMediaStreamSource(stream).connect(analyser);
    dataArray = new Uint8Array(analyser.frequencyBinCount);

    // Setup recorder
    mediaRecorder = new MediaRecorder(stream, { mimeType: 'audio/webm;codecs=opus' });
    audioChunks = [];

    mediaRecorder.ondataavailable = e => {
      if(e.data.size > 0) audioChunks.push(e.data);
    };

    mediaRecorder.onstop = () => {
      audioBlob = new Blob(audioChunks, { type: 'audio/webm' });
      document.getElementById('downloadBtn').disabled = false;
      document.getElementById('uploadBtn').disabled = false;
      document.getElementById('status').textContent =
        `Recording complete! Size: ${(audioBlob.size / 1024).toFixed(2)} KB`;
      stream.getTracks().forEach(track => track.stop());
      if(audioContext) audioContext.close();
      cancelAnimationFrame(animationId);
    };

    mediaRecorder.start();
    startTime = Date.now();
    timerInterval = setInterval(updateTimer, 1000);
    animateWaveform();

    document.getElementById('startBtn').disabled = true;
    document.getElementById('stopBtn').disabled = false;
    document.getElementById('status').textContent = 'Recording...';
    document.getElementById('status').classList.add('recording');

  } catch(err) {
    document.getElementById('status').textContent =
      `Error: ${err.message}. Please allow microphone access.`;
  }
}

// Stop recording
function stopRecording() {
  if(mediaRecorder && mediaRecorder.state !== 'inactive') {
    mediaRecorder.stop();
    clearInterval(timerInterval);
    document.getElementById('startBtn').disabled = false;
    document.getElementById('stopBtn').disabled = true;
    document.getElementById('status').classList.remove('recording');
  }
}

// Download audio file
function downloadAudio() {
  if(audioBlob) {
    const url = URL.createObjectURL(audioBlob);
    const a = document.createElement('a');
    a.href = url;
    a.download = `recording_${Date.now()}.webm`;
    a.click();
    URL.revokeObjectURL(url);
    document.getElementById('status').textContent = 'Audio downloaded!';
  }
}

// Upload audio to Colab workspace
async function uploadToColab() {
  if(!audioBlob) return;

  try {
    document.getElementById('status').textContent = 'Uploading...';
    document.getElementById('uploadBtn').disabled = true;

    const reader = new FileReader();
    reader.readAsDataURL(audioBlob);
    reader.onloadend = async function() {
      const base64data = reader.result.split(',')[1];
      await google.colab.kernel.invokeFunction('notebook.save_audio', [base64data], {});
      document.getElementById('status').textContent =
        '✅ Uploaded as "recorded_audio.wav"!';

      setTimeout(() => {
        document.getElementById('timer').textContent = '00:00';
        document.getElementById('downloadBtn').disabled = false;
        document.getElementById('uploadBtn').disabled = false;
      }, 2000);
    };
  } catch(err) {
    document.getElementById('status').textContent = `Upload error: ${err.message}`;
    document.getElementById('uploadBtn').disabled = false;
  }
}

// Cleanup on page unload
window.addEventListener('beforeunload', () => {
  if(mediaRecorder && mediaRecorder.state === 'recording') mediaRecorder.stop();
});
</script>
"""))

# Python Backend: Save Audio Function
def save_audio(base64_audio):
    """Convert and save uploaded audio as WAV file"""
    try:
        # Decode and save temporary webm file
        audio_data = b64decode(base64_audio)
        with open('temp_audio.webm', 'wb') as f:
            f.write(audio_data)

        # Convert to WAV (16kHz mono)
        audio = AudioSegment.from_file('temp_audio.webm', format='webm')
        audio.export('recorded_audio.wav', format='wav',
                    parameters=['-ar', '16000', '-ac', '1'])

        # Cleanup
        os.remove('temp_audio.webm')
        print("✅ Audio saved as 'recorded_audio.wav' (16kHz mono)")
        return "Success"
    except Exception as e:
        print(f"❌ Error: {e}")
        return "Error"

# Register callback
output.register_callback('notebook.save_audio', save_audio)

print("\n✅ Audio recorder ready! Click 'Start Recording' above.")


🎤 Setting Up Audio Recording Interface



✅ Audio recorder ready! Click 'Start Recording' above.
✅ Audio saved as 'recorded_audio.wav' (16kHz mono)


## Speech-to-Text (STT)

In this section, we will:
- **Load Whisper AI model** - Initialize a speech recognition model (optimized for GPU/CPU)
- **Process audio file** - Convert the recorded audio into text
- **Extract transcript** - Get the spoken words as readable text

**Goal:** Convert voice input into text that can be processed by the AI assistant.

In [9]:
# SPEECH-TO-TEXT (STT)


audio_path = "recorded_audio.wav"

# Setup device
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

print(f"Loading Whisper model on {device}...")
model_stt = WhisperModel("small", device=device, compute_type=compute_type)

# Transcribe (English only)
print(f"Transcribing {audio_path} (English)...")
segments, _ = model_stt.transcribe(
    audio_path,
    beam_size=5,
    language="en"  # Force English language
)
transcript = " ".join([s.text.strip() for s in segments])

# Show result
print("\n📝 Transcript:")
print("-" * 60)
print(transcript)
print("-" * 60)

Loading Whisper model on cuda...


config.json: 0.00B [00:00, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

Transcribing recorded_audio.wav (English)...

📝 Transcript:
------------------------------------------------------------
Hi, Metahuman. My name is Fadi Muteir and I'm 24 years old. Can you help me in some tasks?
------------------------------------------------------------


## Large Language Model (LLM) Setup

In this section, we will:
- **Detect available hardware** - Check if GPU is available for faster processing
- **Load AI language model** - Try loading the best available model based on system resources
- **Configure text generation** - Set up the model for generating intelligent responses
- **Fallback strategy** - Automatically try smaller models if larger ones fail to load

**Goal:** Initialize a powerful language model that can understand questions and generate human-like responses.

In [10]:
# LLM - Mistral 7B or Qwen 2.5 3B


print("=" * 60)
print(" Loading Premium Model (No Auth Required)")
print("=" * 60)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n Device: {device.upper()}")

if device == "cuda":
    print(f" GPU: {torch.cuda.get_device_name(0)}")
    print(f" Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

print("\n Loading model...")

try:
    # Try Mistral 7B first (best quality)
    model_name = "mistralai/Mistral-7B-Instruct-v0.3"
    print("Trying: Mistral 7B Instruct...")

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model_llm = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto",
        low_cpu_mem_usage=True
    )

    qa_pipe = pipeline(
        "text-generation",
        model=model_llm,
        tokenizer=tokenizer,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.15
    )

    model_type = "mistral"
    print(" Mistral 7B loaded!")

except Exception as e:
    print(f" Mistral needs more memory, trying Qwen 2.5 3B...")

    try:
        model_name = "Qwen/Qwen2.5-3B-Instruct"

        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        model_llm = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto",
            trust_remote_code=True,
            low_cpu_mem_usage=True
        )

        qa_pipe = pipeline(
            "text-generation",
            model=model_llm,
            tokenizer=tokenizer,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7
        )

        model_type = "qwen"
        print(" Qwen 2.5 3B loaded!")

    except Exception as e2:
        print(f" Trying smaller model (Qwen 1.5B)...")

        model_name = "Qwen/Qwen2.5-1.5B-Instruct"
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        model_llm = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto",
            trust_remote_code=True,
            low_cpu_mem_usage=True
        )

        qa_pipe = pipeline(
            "text-generation",
            model=model_llm,
            tokenizer=tokenizer,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7
        )

        model_type = "qwen"
        print(" Qwen 2.5 1.5B loaded!")

print("\n" + "=" * 60)
print(f" Model Ready: {model_name.split('/')[-1]}")
print(f" Type: {model_type}")
print("=" * 60)

 Loading Premium Model (No Auth Required)

 Device: CUDA
 GPU: Tesla T4
 Memory: 15.83 GB

 Loading model...
Trying: Mistral 7B Instruct...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


 Mistral 7B loaded!

 Model Ready: Mistral-7B-Instruct-v0.3
 Type: mistral


## Retrieval-Augmented Generation (RAG)

In this section, we will:
- **Analyze the user's question** - Determine if it needs information retrieval or is just conversational
- **Search the knowledge base** - Find relevant information chunks using vector similarity
- **Generate intelligent responses** - Use the LLM to create natural answers based on retrieved context
- **Handle different modes** - Switch between RAG mode (fact-based) and conversational mode (chat)

**Goal:** Create an intelligent Q&A system that retrieves relevant information and generates accurate, natural responses.

In [11]:
# RETRIEVAL-AUGMENTED GENERATION (RAG)

# Helper Functions
def retrieve_top_k(query, k=4, relevance_threshold=0.25):
    """Retrieve top-k relevant chunks from vector database"""
    try:
        q_emb = embedder.encode([query], convert_to_numpy=True).astype("float32")
        distances, indices = index.search(q_emb, k)

        results = []
        for i, (dist, idx) in enumerate(zip(distances[0], indices[0])):
            score = float(1 / (1 + dist))
            if score >= relevance_threshold:
                results.append({
                    "rank": i + 1,
                    "chunk": chunks[idx],
                    "score": score
                })
        return results
    except Exception as e:
        print(f" Retrieval error: {e}")
        return []


def is_conversational(query):
    """Check if query is conversational (no retrieval needed)"""
    patterns = ["hi", "hello", "hey", "my name", "i am", "help me",
                "can you", "hear me", "who are you", "how are you"]
    return any(pattern in query.lower() for pattern in patterns)


def create_prompt(query, context, model_name, model_type):
    """Create model-specific prompts for concise answers"""

    # Mistral models
    if "mistral" in model_name.lower() or model_type == "mistral":
        if context:
            return f"""<s>[INST] You are a helpful assistant. Answer in 1-2 sentences using this context.

Context: {context}

Question: {query} [/INST]"""
        else:
            return f"""<s>[INST] You are a friendly assistant. Respond naturally in 1-2 sentences.

{query} [/INST]"""

    # Qwen models
    elif "qwen" in model_name.lower() or model_type == "qwen":
        if context:
            return f"""<|im_start|>system
You are a helpful assistant. Answer questions concisely in 1-2 sentences using the provided context.<|im_end|>
<|im_start|>user
Context: {context}

Question: {query}<|im_end|>
<|im_start|>assistant
"""
        else:
            return f"""<|im_start|>system
You are a friendly assistant. Respond naturally in 1-2 sentences.<|im_end|>
<|im_start|>user
{query}<|im_end|>
<|im_start|>assistant
"""

    # Gemma models
    elif "gemma" in model_name.lower() or model_type == "gemma":
        if context:
            return f"""<start_of_turn>user
Answer in 1-2 sentences using this context:

Context: {context}

Question: {query}<end_of_turn>
<start_of_turn>model
"""
        else:
            return f"""<start_of_turn>user
{query}

Respond in 1-2 sentences.<end_of_turn>
<start_of_turn>model
"""

    # StableLM models
    elif "stablelm" in model_name.lower() or model_type == "stablelm":
        if context:
            return f"""<|system|>
Answer in 1-2 sentences using the context.<|endoftext|>
<|user|>
Context: {context}

{query}<|endoftext|>
<|assistant|>
"""
        else:
            return f"""<|system|>
Respond naturally in 1-2 sentences.<|endoftext|>
<|user|>
{query}<|endoftext|>
<|assistant|>
"""

    # Phi-3/3.5
    elif "phi" in model_name.lower():
        if context:
            return f"""<|system|>
Answer in 1-2 short sentences. No lists or examples.<|end|>
<|user|>
Context: {context}

{query}<|end|>
<|assistant|>"""
        else:
            return f"""<|system|>
Respond naturally in 1-2 short sentences.<|end|>
<|user|>
{query}<|end|>
<|assistant|>"""

    # Llama models
    elif "llama" in model_name.lower():
        if context:
            return f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Answer in 1-2 sentences only.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: {context}

{query}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
        else:
            return f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Reply in 1-2 sentences.<|eot_id|><|start_header_id|>user<|end_header_id|>

{query}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

    # Default/TinyLlama
    else:
        if context:
            return f"Context: {context}\n\nQuestion: {query}\n\nAnswer in 1-2 sentences:"
        else:
            return f"{query}\n\nRespond in 1-2 sentences:"


def extract_answer(full_response, prompt, model_name, model_type):
    """Extract clean answer based on model type"""

    # Mistral
    if "mistral" in model_name.lower() or model_type == "mistral":
        if "[/INST]" in full_response:
            answer = full_response.split("[/INST]")[-1].strip()
        else:
            answer = full_response[len(prompt):].strip()
        answer = answer.replace("</s>", "").strip()

    # Qwen
    elif "qwen" in model_name.lower() or model_type == "qwen":
        if "<|im_start|>assistant" in full_response:
            answer = full_response.split("<|im_start|>assistant")[-1].strip()
        else:
            answer = full_response[len(prompt):].strip()
        answer = answer.replace("<|im_end|>", "").replace("<|endoftext|>", "").strip()

    # Gemma
    elif "gemma" in model_name.lower() or model_type == "gemma":
        if "<start_of_turn>model" in full_response:
            answer = full_response.split("<start_of_turn>model")[-1].strip()
        else:
            answer = full_response[len(prompt):].strip()
        answer = answer.replace("<end_of_turn>", "").strip()

    # StableLM
    elif "stablelm" in model_name.lower() or model_type == "stablelm":
        if "<|assistant|>" in full_response:
            answer = full_response.split("<|assistant|>")[-1].strip()
        else:
            answer = full_response[len(prompt):].strip()
        answer = answer.replace("<|endoftext|>", "").strip()

    # Phi
    elif "phi" in model_name.lower():
        if "<|assistant|>" in full_response:
            answer = full_response.split("<|assistant|>")[-1].strip()
        else:
            answer = full_response[len(prompt):].strip()
        answer = answer.replace("<|end|>", "").replace("</s>", "").strip()

    # Llama
    elif "llama" in model_name.lower():
        if "<|start_header_id|>assistant<|end_header_id|>" in full_response:
            answer = full_response.split("<|start_header_id|>assistant<|end_header_id|>")[-1].strip()
        else:
            answer = full_response[len(prompt):].strip()
        answer = answer.replace("<|eot_id|>", "").replace("<|end_of_text|>", "").strip()

    # Default
    else:
        answer = full_response[len(prompt):].strip()

    return answer


def truncate_response(text, max_sentences=2):
    """Force truncate to maximum sentences"""
    import re
    sentences = re.split(r'[.!?]+', text)
    sentences = [s.strip() for s in sentences if s.strip()]

    if len(sentences) > max_sentences:
        result = '. '.join(sentences[:max_sentences]) + '.'
    elif sentences:
        result = '. '.join(sentences)
        if not result.endswith(('.', '!', '?')):
            result += '.'
    else:
        result = text

    # Remove numbered lists
    lines = result.split('\n')
    clean_lines = []
    for line in lines:
        line = line.strip()
        if line and not (line[0].isdigit() or line[0] in ['•', '-', '*', ')']):
            clean_lines.append(line)

    result = ' '.join(clean_lines) if clean_lines else result

    return result


# Main RAG Pipeline

print("=" * 60)
print("🔍 RAG Pipeline - Processing Query")
print("=" * 60)

# 1. Get Query
try:
    query = transcript.strip() if 'transcript' in locals() and transcript.strip() else "What is virtual reality?"
except NameError:
    query = "What is virtual reality?"

print(f"\n Query: {query}\n")

# 2. Determine Mode
is_conv = is_conversational(query)
context = None

if not is_conv:
    print(" RAG Mode: Searching knowledge base...")
    retrieved = retrieve_top_k(query, k=3, relevance_threshold=0.25)

    if retrieved:
        print(f" Found {len(retrieved)} relevant chunks")
        context = "\n\n".join([r["chunk"] for r in retrieved])
    else:
        print("  No relevant context found")
else:
    print("  Conversational Mode: No retrieval needed")

# 3. Create Prompt
prompt = create_prompt(query, context, model_name, model_type)
print(f"\n Prompt: {len(prompt)} characters")

# 4. Generate Response
print("\n Generating response...\n")

try:
    response = qa_pipe(
        prompt,
        max_new_tokens=80,
        min_new_tokens=10,
        do_sample=True,
        temperature=0.6,
        top_p=0.85,
        repetition_penalty=1.2,
        pad_token_id=qa_pipe.tokenizer.eos_token_id,
        eos_token_id=qa_pipe.tokenizer.eos_token_id,
        use_cache=False,
        no_repeat_ngram_size=3
    )

    full_response = response[0]["generated_text"]

    # Extract clean answer
    answer_text = extract_answer(full_response, prompt, model_name, model_type)

    # Truncate to 2 sentences
    answer_text = truncate_response(answer_text, max_sentences=2)

    # Display Result
    print("=" * 60)
    print(" AI RESPONSE")
    print("=" * 60)
    print(answer_text)
    print("=" * 60)

except Exception as e:
    print(f" Generation error: {e}")
    import traceback
    traceback.print_exc()
    answer_text = "Sorry, I encountered an error."

🔍 RAG Pipeline - Processing Query

 Query: Hi, Metahuman. My name is Fadi Muteir and I'm 24 years old. Can you help me in some tasks?

  Conversational Mode: No retrieval needed

 Prompt: 175 characters

 Generating response...

 AI RESPONSE
Hello Fadi. Of course, I can assist with various tasks such as answering questions, setting reminders, providing information on topics of interest, or even helping brainstorm ideas for your projects.


## Text-to-Speech (TTS)

In this section, we will:
- **Load TTS model** - Initialize a text-to-speech engine (Coqui TTS)
- **Convert text to speech** - Transform the AI's text response into natural-sounding audio
- **Generate audio file** - Create a WAV file that can be played back
- **Display audio player** - Show an embedded audio player in the notebook

**Goal:** Convert the AI's written response into spoken audio, completing the voice assistant cycle.

In [19]:
# CLEAR MALE VOICE - USING GTTS (Google Text-to-Speech)

print("=" * 60)
print(" Generating Clear Male Voice")
print("=" * 60)

# Install gTTS if needed
import subprocess
subprocess.run(['pip', 'install', '-q', 'gtts', 'pydub'],
               stdout=subprocess.DEVNULL,
               stderr=subprocess.DEVNULL)

# Configuration
output_path = "out/answer.wav"
Path("out").mkdir(exist_ok=True)

print("\n Generating speech with Google TTS...")

# Use Google Text-to-Speech (Free, High Quality, No API Key)
from gtts import gTTS

# Generate speech
tts = gTTS(
    text=answer_text,
    lang='en',
    slow=False,
    tld='com.au'  # Australian accent (deeper, more masculine)
)

# Save as MP3 first
temp_mp3 = "out/temp.mp3"
tts.save(temp_mp3)
print(" Speech generated")

# Convert MP3 to WAV and enhance
print("\n Converting and enhancing audio...")

from pydub import AudioSegment

# Load MP3
audio = AudioSegment.from_mp3(temp_mp3)

# Enhance audio quality
# 1. Increase volume (make it loud)
audio = audio + 8  # Increase by 8dB

# 2. Normalize
audio = audio.normalize()

# 3. Apply high-pass filter (remove low noise)
audio = audio.high_pass_filter(80)

# 4. Apply compression (make voice clearer and more consistent)
audio = audio.compress_dynamic_range()

# Export as high-quality WAV
audio.export(
    output_path,
    format="wav",
    parameters=[
        "-ar", "44100",  # High sample rate
        "-ac", "1",       # Mono (clearer for voice)
    ]
)

# Clean up temp file
import os
os.remove(temp_mp3)

print("✅ Audio enhanced - clear and loud")

# Audio Info
audio_data, sr = librosa.load(output_path, sr=None)
duration = librosa.get_duration(y=audio_data, sr=sr)

# Play Audio
print("\n Audio Player:")
display(Audio(output_path, autoplay=False))

print("\n" + "=" * 60)
print("✅ DONE")
print("=" * 60)


 Generating Clear Male Voice

 Generating speech with Google TTS...
 Speech generated

 Converting and enhancing audio...
✅ Audio enhanced - clear and loud

 Audio Player:



✅ DONE
